This file is part of BrainMolecularAtlas.

This file make figure 10 (metabolites)

Copyright (c) 2021 Blue Brain Project/EPFL 

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.
This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [1]:
import numpy as np
import pandas as pd
import re

import xlrd
import xlsxwriter

import altair as alt
import altair_catplot as altcat


pd.set_option('display.max_columns',100)

In [ ]:
# To reproduce figures:

In [2]:
df = pd.ExcelFile('Data Sheet 5.XLSX') # Supplementary Materials Data Sheet 5.XLSX
df = df.parse('Sheet1')
df = df.drop(columns='Unnamed: 0')
df.head()

,Methods,Organism,age,cellType,cid_pubchem,comments,level_units,level_units_ini,level_units_iniUniformNames,level_value_uM,locLevelOfDetails,location,methods details,molecule_name,organism/cell culture,reference,subcellularLoc,tissue
0,mass spectrometry,mouse,8-week-old,NaN,7017974,cerebra mouse 1 known,uM,nmol/g,umol/g,0.002649,tissue,cerebra mouse 1 known,ce-tofms,(2s)-2-(tert-butoxycarbonylamino)-4-oxo-4-(9h-...,mouse,"Sugimoto, 2012, Nucleic Acids Research, MMMDB:...",whole cell,brain
1,mass spectrometry,mouse,8-week-old,NaN,7017974,cerebra mouse 2 known,uM,nmol/g,umol/g,0.002752,tissue,cerebra mouse 2 known,ce-tofms,(2s)-2-(tert-butoxycarbonylamino)-4-oxo-4-(9h-...,mouse,"Sugimoto, 2012, Nucleic Acids Research, MMMDB:...",whole cell,brain
2,mass spectrometry,mouse,8-week-old,NaN,7017974,cerebella mouse 2 known,uM,nmol/g,umol/g,0.005905,tissue,cerebella mouse 2 known,ce-tofms,(2s)-2-(tert-butoxycarbonylamino)-4-oxo-4-(9h-...,mouse,"Sugimoto, 2012, Nucleic Acids Research, MMMDB:...",whole cell,brain
3,mass spectrometry,mouse,8-week-old,NaN,7017974,cerebella mouse 1 known,uM,nmol/g,umol/g,0.007253,tissue,cerebella mouse 1 known,ce-tofms,(2s)-2-(tert-butoxycarbonylamino)-4-oxo-4-(9h-...,mouse,"Sugimoto, 2012, Nucleic Acids Research, MMMDB:...",whole cell,brain
4,mass spectrometry,mouse,8-week-old,NaN,440129,cerebra mouse 1 known,uM,nmol/g,umol/g,0.001362,tissue,cerebra mouse 1 known,ce-tofms,(2s)-2-hydroxy-3-(1h-imidazol-5-yl)propanoic acid,mouse,"Sugimoto, 2012, Nucleic Acids Research, MMMDB:...",whole cell,brain


In [3]:
df = df.rename(columns={'methods_brief':'Methods'})

In [4]:
df.columns

Index(['Methods', 'Organism', 'age', 'cellType', 'cid_pubchem', 'comments',
       'level_units', 'level_units_ini', 'level_units_iniUniformNames',
       'level_value_uM', 'locLevelOfDetails', 'location', 'methods details',
       'molecule_name', 'organism/cell culture', 'reference', 'subcellularLoc',
       'tissue'],
      dtype='object')

In [5]:
df['log_level_value'] = np.log(df['level_value_uM'].astype(np.float64))

mrsmol = df['molecule_name'][df['Methods']=='mrs'].unique()
msmol = df['molecule_name'][df['Methods']=='mass spectrometry '].unique()
msmolmrsmol = list(set(msmol).intersection(mrsmol))

df['Organism'] = df['organism/cell culture'].copy()

df['Organism'][df['organism/cell culture'].isin(['human hela cells','human hct116 colon cancer cells ',
       'human glioblastoma ln229 cells in high glucose conditions',
       'human glioblastoma ln229 cells in low glucose conditions'])] = 'human'

In [8]:
chart = alt.Chart(df[(df['molecule_name'].isin(msmolmrsmol))&(df['Methods']!='database')&(df['Organism'].isin(['rat','mouse','human','human hela cells','review', 'cultured astrocytes']))].sort_values(by=['Organism']),
         width=500,
         height=500
         ).mark_point().configure_axis(labelLimit=1000).configure_legend(labelLimit=1000).encode(
    x=alt.X('log_level_value:Q', title = 'log(Concentration, uM)'),
    y=alt.Y('molecule_name:N', title=None),
    color=alt.Color('Organism:N',legend=alt.Legend(title="Organism")),
    shape=alt.Shape('Methods',legend=alt.Legend(title="Methods"))
).configure_axis(labelFontSize=14,titleFontSize=14,tickSize=12,labelLimit=1000).configure_legend(labelFontSize=14,titleFontSize=14,labelLimit=1000)

#chart.save('fig10A.html')

In [9]:
# mouse ms mrs

chart=alt.Chart(df[(df['molecule_name'].isin(msmolmrsmol))&(df['Methods'].isin(['mrs']))],
         width=500,
         height=500
         ).mark_point().configure_axis(labelLimit=1000).configure_legend(labelLimit=1000).encode(
    x=alt.X('log_level_value:Q', title = 'log(Concentration, uM)'),
    y=alt.Y('molecule_name:N', title=None),
    color=alt.Color('Organism:N',legend=alt.Legend(title="Organism")),
    #shape=alt.Shape('Organism',legend=alt.Legend(title="Organism"))
).configure_axis(labelFontSize=14,titleFontSize=14,tickSize=12,labelLimit=1000).configure_legend(labelFontSize=14,titleFontSize=14,labelLimit=1000)


#chart.save('fig10B_left.html')


In [10]:
# mouse ms mrs

chart=alt.Chart(df[(df['molecule_name'].isin(msmolmrsmol))&(df['Methods'].isin(['mass spectrometry ']))],
         width=500,
         height=500
         ).mark_point().configure_axis(labelLimit=1000).configure_legend(labelLimit=1000).encode(
    x=alt.X('log_level_value:Q', title = 'log(Concentration, uM)'),
    y=alt.Y('molecule_name:N', title=None),
    color=alt.Color('Organism:N',legend=alt.Legend(title="Organism")),
    #shape=alt.Shape('Organism',legend=alt.Legend(title="Organism"))
).configure_axis(labelFontSize=14,titleFontSize=14,tickSize=12,labelLimit=1000).configure_legend(labelFontSize=14,titleFontSize=14,labelLimit=1000)


#chart.save('fig10B_right.html')


In [ ]:
# Data prep

In [2]:
# For code on data-search see BrainMolecularAtlas/figS7_adj_conc_auto_2_plots.R
# The code below is for data prep. However, full data prep involves some manual steps, 
# so that to reproduce the figures use the data from the supplementary Data Sheet 5.XLSX 
# and scroll up to "To reproduce figures:")


In [3]:
cf = pd.ExcelFile("/Users/polina/Documents/BBP_project/MyPaper/molAtlasTables/concentrations_v2.xlsx") # raw data from referenced sources just being put together in one file
df = cf.parse("Sheet1")

s = df['level_value'].str.split('\(|\)|\;|\:').apply(pd.Series, 1).stack()
s.index = s.index.droplevel(-1)
s.name = 'level_value'
del df['level_value']

df = df.join(s)
df['level_value'].replace('', np.nan, inplace=True)
df.dropna(subset=['level_value'], inplace=True)
df[['level_value']] = df[['level_value']].astype(float)
df['level_units_ini'] = df['level_units'].copy()


# Convert diff units to uM:

df['level_value'][df['level_units']=='mM'] = df['level_value'][df['level_units']=='mM']*1000.0
df['level_units'].loc[df['level_units']=='mM'] = 'uM'


df['level_value'].loc[df['level_units']=='nM'] = df['level_value'].loc[df['level_units']=='nM']*0.001
df['level_units'].loc[df['level_units']=='nM'] = 'uM'

df['level_value'].loc[df['level_units']=='umol/mL'] = df['level_value'].loc[df['level_units']=='umol/mL']*1000.0  
df['level_units'].loc[df['level_units']=='umol/mL'] = 'uM'


df['level_units'].loc[df['level_units']=='umol'] = 'uM'
df['level_units'].loc[df['level_units']=='umol/L'] = 'uM' 


df['level_value'].loc[df['level_units']=='nmol/g'] = df['level_value'].loc[df['level_units']=='nmol/g']*0.001
df['level_units'].loc[df['level_units']=='nmol/g'] = 'umol/g'
df['level_units'].loc[df['level_units']=='microMol/g'] = 'umol/g'
df['level_units'].loc[df['level_units']=='mmol/kg ww'] = 'umol/g'
df['level_value'].loc[df['level_units']=='ug/g tissue weight'] = df['level_value'].loc[df['level_units']=='ug/g tissue weight']*1000.0
df['level_units'].loc[df['level_units']=='ng/g tissue weight'] = 'ng/g wet tissue'
df['level_units'].loc[df['level_units']=='ug/g tissue weight'] = 'ng/g wet tissue'


df['molecule_abbreviation'].loc[df['level_units']=='ng/g wet tissue'].unique()  #array(['AdenineP', 'Ala', 'Cytosine', 'DA', 'EP', 'GABA', 'Gln', 'Gly','guanine', 'Histamine', 'histidine', 'HX', 'inosine', 'Asn', 'Asp','Cys', 'Glu', 'Hpro', 'Ile', 'Leu', 'Met', 'Orn', 'Phe', 'Ser','Thr', 'Trp', 'Tyr', 'Val', 'Arg', 'Pro', 'Levo', 'Lys', 'NE','5-HT', 'UricAcid', 'xanthine', 'GLU', 'BH4'], dtype=object)

mw38 = pd.DataFrame([['5-HT',176.219], ['AdenineP',233.124], ['Ala',89.094], ['Arg',174.204], ['Asn',132.119], ['Asp',133.103], ['BH4',241.251], ['Cys',121.154],
       ['Cytosine',111.104], ['DA',153.181], ['EP',183.207], ['GABA',103.121], ['Gln',146.146], ['Glu',147.13], ['GLU',147.13], ['Gly',75.067],
       ['guanine',151.129], ['Histamine',111.148], ['histidine',155.157], ['Hpro',131.131], ['HX',136.114], ['Ile',131.175],
       ['inosine',268.229], ['Leu',131.175], ['Levo',197.19], ['Lys',146.19], ['Met',149.208], ['NE',169.18], ['Orn',132.163], ['Phe',165.192], ['Pro',115.132],
       ['Ser',105.093], ['Thr',119.12], ['Trp',204.229], ['Tyr',181.191], ['UricAcid',168.112], ['Val',117.148], ['xanthine',152.113]],columns=['molecule_abbreviation','MW_g/mol'])


# rat brain density: 1.04-1.05 g/ml https://link.springer.com/chapter/10.1007/978-3-7091-9115-6_12
# human brain density: 1.03 kg/l

df = pd.merge(df,mw38,how='left',on='molecule_abbreviation')

df['level_value'].loc[df['level_units']=='ng/g wet tissue'] = df['level_value'].loc[df['level_units']=='ng/g wet tissue']*1.04/df['MW_g/mol'] # 1.04 g/ml density
df['level_units'].loc[df['level_units']=='ng/g wet tissue'] = 'uM'

df['level_value'].loc[df['level_units']=='nmol/mg dry weight'] = df['level_value'].loc[df['level_units']=='nmol/mg dry weight']*0.2 # 80% water
df['level_units'].loc[df['level_units']=='nmol/mg dry weight'] = 'umol/g'

df['level_value'].loc[df['level_units']=='umol/g'] = df['level_value'].loc[df['level_units']=='umol/g']*1.04 # 1.04 g/ml density
df['level_units'].loc[df['level_units']=='umol/g'] = 'uM'

df['level_value'].loc[df['level_units']=='ug/mol'] = df['level_value'].loc[df['level_units']=='ug/mol']*1.04 # fig2,3 Kulak et al. ug/mol -> umol/g # 1.04 g/ml density
df['level_units'].loc[df['level_units']=='ug/mol'] = 'uM'



# Nomenclature:

df['molecule_abbreviation'].loc[df['molecule_abbreviation']=='Glu'] = 'GLU'
df['molecule_abbreviation'].loc[df['molecule_abbreviation']=='Gln'] = 'GLN'

gb = df.groupby(['molecule_abbreviation']).size().to_frame(name='m_counts').reset_index()
df2 = pd.merge(df,gb,on='molecule_abbreviation',how='left')
df2 = df2.loc[df2['level_units']=='uM']

## Get data from PubChem:
df["molecule_name"] = df["molecule_name"].str.lower()
cs = pcp.get_compounds(df['molecule_name_4search'][1], 'name')
df4 = pcp.compounds_to_frame(cs, properties=['iupac_name', 'molecular_weight', 'xlogp','synonyms'])
df4['molecule_abbreviation'] = df['molecule_name'][0]
df4 = df4.reset_index()

pubchemdf = pd.DataFrame(columns=['iupac_name', 'molecular_weight', 'synonyms', 'xlogp','molecule_name'])
for i,mol in enumerate(df['molecule_name'].unique()):
    cs = pcp.get_compounds(mol, 'name')
    df4 = pcp.compounds_to_frame(cs, properties=['iupac_name', 'molecular_weight', 'xlogp','synonyms'])
    df4['molecule_name'] = mol
    pubchemdf = pubchemdf.append(df4,sort=False)

pubchemdf = pubchemdf.reset_index(drop=True)
pubchemdf = pubchemdf.drop("index", axis=1)
pubchemdf = pubchemdf.drop("molecule_name_4search", axis=1)
pubchemdf = pubchemdf.rename(columns={'molecule_abbreviation': 'molecule_name_4search'})

dfp = pd.merge(df, pubchemdf, how='left', on='molecule_name_4search')

# man refine
dfp = dfp.drop('synonyms', 1)
dfp = dfp.drop('MW_g/mol', 1)
dfp = dfp.drop('iupac_name', 1)
dfp = dfp.drop_duplicates()

occurence = dfp['molecule_name'].value_counts().rename('occurence')

dfp = dfp.merge(occurence.to_frame(),
                                left_on='molecule_name',
                                right_index=True)

dfp["molecule_name_4search"] = dfp["molecule_name_4search"].str.lower()
dfp['log_level_value'] = np.log(dfp['level_value'])

# man refine
sd = dfp.groupby('molecule_name')['level_value'].agg(np.std, ddof=1).rename('SD')
dfp = dfp.merge(sd.to_frame(),
                                left_on='molecule_name',
                                right_index=True)

#writer = pd.ExcelWriter('conc_part1.xlsx', engine='xlsxwriter')
#dfp.to_excel(writer, 'Sheet1')
#writer.save()


# extra data from Zheng 2016 Scientific reports
cf = pd.ExcelFile("/Users/polina/Documents/BBP_project/MyPaper/molAtlasTables/concentrations_v2.xlsx") # raw data from referenced sources just being put together in one file
df = cf.parse("Sheet2") # Sheet2 is Zheng, 2016, Scientific reports data

df['level_value'] = df['level_value'].str.split('±').str[0]
df['level_value'] = pd.to_numeric(df['level_value'])

pubchemdf = pd.DataFrame(columns=['iupac_name', 'molecular_weight', 'synonyms', 'xlogp','molecule_name'])
for i,mol in enumerate(df['molecule_name_4search'].unique()):
    cs = pcp.get_compounds(mol, 'name')
    df4 = pcp.compounds_to_frame(cs, properties=['iupac_name', 'molecular_weight', 'xlogp','synonyms'])
    df4['molecule_name_4search'] = mol
    pubchemdf = pubchemdf.append(df4,sort=False)

    
df['level_value'].loc[df['level_units']=='nmol/g brain tissue'] = df['level_value'].loc[df['level_units']=='nmol/g brain tissue']*0.001
df['level_units'].loc[df['level_units']=='nmol/g brain tissue'] = 'umol/g'
df['level_value'].loc[df['level_units']=='umol/g'] = df['level_value'].loc[df['level_units']=='umol/g']*1.04
df['level_units'].loc[df['level_units']=='umol/g'] = 'uM'
df['level_units_ini'] = 'nmol/g brain tissue'
sd = df.groupby('molecule_name_4search')['level_value'].agg(np.std, ddof=1).rename('SD')
df = df.merge(sd.to_frame(),
                                left_on='molecule_name_4search',
                                right_index=True)

df = df.drop("molecule_name", axis=1)
df = df.rename(columns=({"molecule_name_4search":"molecule_name"}))



# concat data
df3 = pd.concat([df,dfp])
occurence = df3['molecule_name'].value_counts().rename('occurence')
df3 = df3.merge(occurence.to_frame(),
                                left_on='molecule_name',
                                right_index=True)

df3['log_level_value'] = np.log(df3['level_value'])
df3 = df3.reset_index()
df3 = df3.drop('index', axis=1) 

df3['molecule_name'] = df3['molecule_name'].str.lower()
df3['level_units_iniUniformNames'] = df3['level_units_ini'].copy()
df3['level_units_iniUniformNames'].loc[df3['level_units_iniUniformNames']=='umol'] = 'uM'
df3['level_units_iniUniformNames'].loc[df3['level_units_iniUniformNames']=='umol/L'] = 'uM'
df3['level_units_iniUniformNames'].loc[df3['level_units_iniUniformNames']=='microMol/g'] = 'umol/g'
df3['level_units_iniUniformNames'].loc[df3['level_units_iniUniformNames']=='mmol/kg ww'] = 'umol/g'
df3['level_units_iniUniformNames'].loc[df3['level_units_iniUniformNames']=='ng/g tissue weight'] = 'ng/g wet tissue'


df3['level_units_iniUniformNames'].loc[df3['level_units_iniUniformNames']=='mM'] = 'uM'
df3['level_units_iniUniformNames'].loc[df3['level_units_iniUniformNames']=='nM'] = 'uM'
df3['level_units_iniUniformNames'].loc[df3['level_units_iniUniformNames']=='umol/mL'] = 'uM'
df3['level_units_iniUniformNames'].loc[df3['level_units_iniUniformNames']=='nmol/g'] = 'umol/g'
df3['level_units_iniUniformNames'].loc[df3['level_units_iniUniformNames']=='ug/g tissue weight'] = 'ng/g wet tissue'
df3['level_units_iniUniformNames'].loc[df3['level_units_iniUniformNames']=='ug/mol'] = 'umol/g' # see comment above about Fig source


df3['log_level_value'] = df3['log_level_value'].replace([np.inf, -np.inf], np.nan) # not interested in 0



df3['locLevelOfDetails'] = df3['locLevelOfDetails'].str.lower()
df3['organism/cell culture'] = df3['organism/cell culture'].str.lower()
df3['tissue'] = df3['tissue'].str.lower()
df3['cellType'] = df3['cellType'].str.lower()
df3['subcellularLoc'] = df3['subcellularLoc'].str.lower()
df3['sex'] = df3['sex'].str.lower()
df3['age'] = df3['age'].str.lower()
df3['methods_brief'] = df3['methods_brief'].str.lower()
df3['methods'] = df3['methods'].str.lower()

# write to file, man ref, load
#writer = pd.ExcelWriter('conc_v2.xlsx', engine='xlsxwriter')
#df3.to_excel(writer, 'Sheet1')
#writer.save()
df = pd.ExcelFile('conc_v3.xlsx') # man ref
df = df.parse('Sheet1')
df = df.drop(columns='Unnamed: 0')


df['log_level_value'] = np.log(df['level_value'])
df.loc[df['Organism'].isin(['review',
       'model:secondary data',np.nan,'estimation']),'Organism'] = 'not given'
#dfFullStat.head()

df['idOrg'] = df['molecule_name']+'#'+df['Organism']

dfFullStat = df.copy()
dfFullStat = dfFullStat[~dfFullStat['idOrg'].isna()]

# Consider removing these sources as outliers
#df = df[df['reference']!='Tae-Hyun Kim, 2014, Journal of Analytical Methods in Chemistry, Quantification of Neurotransmitters in Mouse Brain Tissue by Using Liquid Chromatography Coupled Electrospray Tandem Mass Spectrometry']
#df = df[df['reference']!='Chen, 2016, Cell, Absolute Quantification of Matrix Metabolites Reveals the Dynamics of Mitochondrial Metabolism']


# methods and organisms effects correction experimenting very basic things
#CV
def coeff_of_var(data):
    return np.std(data) / np.abs(np.mean(data))
def coeff_of_var_log(data):
    return np.sqrt(10**(np.log(10)*(np.std(data)**2)) - 1) # 2017 Correct use of percent coefficient of variation (%CV) formula for log-transformed data https://medcraveonline.com/MOJPB/MOJPB-06-00200.pdf 

mrsmol = df['molecule_name'][df['Methods']=='mrs'].unique()
msmol = df['molecule_name'][df['Methods']=='mass spectrometry '].unique()
msmolmrsmol = list(set(msmol).intersection(mrsmol))

mousemol = df['molecule_name'][df['Organism']=='mouse'].unique()
ratmol = df['molecule_name'][df['Organism']=='rat'].unique()
humanmol = df['molecule_name'][df['Organism']=='human'].unique()

commonOrgMol1 = list(set(mousemol).intersection(ratmol))
commonOrgMol = list(set(commonOrgMol1).intersection(humanmol))

# It would be better to consider only common molecules measured in all organisms with all methods, but such data set will be very small and not significant statistically
cv_method_organismCO = df[df['molecule_name'].isin(commonOrgMol)].groupby(['Methods','Organism']).agg(coeff_of_var).reset_index()
cv_method_organismCO = cv_method_organismCO.drop(['log_level_value','occurence'],axis=1)
cv_method_organismCO

cv_method_organismCOO= df[df['molecule_name'].isin(commonOrgMol)].groupby(['Organism']).agg(coeff_of_var).reset_index()
cv_method_organismCOO = cv_method_organismCOO.drop(['log_level_value','occurence'],axis=1)
cv_method_organismCOO

cv_method_organismCOM= df[df['molecule_name'].isin(commonOrgMol)].groupby(['Methods']).agg(coeff_of_var).reset_index()
cv_method_organismCOM = cv_method_organismCOM.drop(['log_level_value','occurence'],axis=1)
cv_method_organismCOM



dfFullStat['log_level_value_corr'] = dfFullStat['log_level_value'].copy()  #"NaN"
nc =0
for i,molecule_name in enumerate(dfFullStat['molecule_name']):
    if dfFullStat['Organism'][i]=='mouse':
        pass
       
    elif ( (dfFullStat['Organism'][i]=='rat') &(molecule_name in mouse_molecules)):
        
        dfFullStat['log_level_value_corr'][i] = dfFullStat['log_level_value'][i].copy() -np.median(dfFullStat[dfFullStat['Organism']=='rat']['log_level_value']) + np.median(dfFullStat[dfFullStat['Organism']=='mouse']['log_level_value'])
        
    elif ( (dfFullStat['Organism'][i]=='human') &(molecule_name in mouse_molecules)):
        
        dfFullStat['log_level_value_corr'][i] = dfFullStat['log_level_value'][i].copy() -np.median(dfFullStat[dfFullStat['Organism']=='human']['log_level_value']) + np.median(dfFullStat[dfFullStat['Organism']=='mouse']['log_level_value'])
    
    elif ( molecule_name in mouse_molecules):
        
        dfFullStat['log_level_value_corr'][i] = dfFullStat['log_level_value'][i].copy() -np.median(dfFullStat[~dfFullStat['Organism'].isin(['mouse','rat','human'])]['log_level_value']) + np.median(dfFullStat[dfFullStat['Organism']=='mouse']['log_level_value'])
    else:
        #dfFullStat['level_value_corr'][i] = dfFullStat['level_value'][i].copy() 
        #print('not corrected ')  # this case never happens
        nc = nc+1
        #pass
        
dfBC = df[df['molecule_name'].isin(commonOrgMol)]
dfBC['id'] = dfBC['molecule_name']+'#'+dfBC['Organism']




dfBCom = df.groupby(['molecule_name','Organism','Methods'])['level_value'].describe().reset_index()
dfBCom['id2'] = dfBCom['molecule_name']+'#'+dfBCom['Organism']+'#'+dfBCom['Methods']

df['id2'] = df['molecule_name']+'#'+df['Organism']+'#'+df['Methods']

dfFullStatOM = pd.merge(df, dfBCom[['count', 'mean', 'std', 'min','25%', '50%', '75%', 'max', 'id2']], on='id2',how='inner')

dfBCo = df.groupby(['molecule_name','Organism'])['level_value'].describe().reset_index()
dfBCo['idOrg'] = dfBCo['molecule_name']+'#'+dfBCo['Organism']
df['idOrg'] = df['molecule_name']+'#'+df['Organism']

dfFullStat = pd.merge(df, dfBCo[['count', 'mean', 'std', 'min', '25%','50%', '75%', 'max', 'idOrg']], on='idOrg',how='inner')
rat_molecules = []
mouse_molecules = []
human_molecules =[]
na_molecules = []
astr_molecules = []

for i in range(len(dfFullStat)):
    if '#rat' in dfFullStat['idOrg'][i]:
        rat_molecules.append(dfFullStat['molecule_name'][i])
    elif '#mouse' in dfFullStat['idOrg'][i]:
        mouse_molecules.append(dfFullStat['molecule_name'][i])
    elif '#human' in dfFullStat['idOrg'][i]:
        human_molecules.append(dfFullStat['molecule_name'][i])
    elif '#not given' in dfFullStat['idOrg'][i]:
        na_molecules.append(dfFullStat['molecule_name'][i])
    elif '#cultured astrocytes' in dfFullStat['idOrg'][i]:
        astr_molecules.append(dfFullStat['molecule_name'][i])
        
        
        
        
dfcm = dfFullStat[(dfFullStat['molecule_name'].isin(commonOrgMol))]
dfcm = dfcm.reset_index(drop='True')

nc =0

for i,molecule_name in enumerate(dfcm['molecule_name']):
    
    if dfcm['Organism'][i]=='mouse':
        pass
       
    elif ( (dfcm['Organism'][i]=='rat') &(molecule_name in mouse_molecules)):
        
        dfcm['log_level_value_corr'][i] = dfcm['log_level_value'][i].copy() -np.median(dfcm[dfcm['Organism']=='rat']['log_level_value']) + np.median(dfcm[dfcm['Organism']=='mouse']['log_level_value'])
        
    elif ( (dfcm['Organism'][i]=='human') &(molecule_name in mouse_molecules)):
        
        dfcm['log_level_value_corr'][i] = dfcm['log_level_value'][i].copy() -np.median(dfcm[dfcm['Organism']=='human']['log_level_value']) + np.median(dfcm[dfcm['Organism']=='mouse']['log_level_value'])
    
    elif ( molecule_name in mouse_molecules):
        
        dfcm['log_level_value_corr'][i] = dfcm['log_level_value'][i].copy() -np.median(dfcm[~dfcm['Organism'].isin(['mouse','rat','human'])]['log_level_value']) + np.median(dfcm[dfcm['Organism']=='mouse']['log_level_value'])
    else:
        #dfcm['level_value_corr'][i] = dfcm['level_value'][i].copy() 
        #print('not corrected ')  # this case never happens
        nc = nc+1
        #pass
        
dfFullStat['level_value_corr'] = dfFullStat['level_value'].copy()  #"NaN"

for i,molecule_name in enumerate(dfFullStat['molecule_name']):
    if dfFullStat['Organism'][i]=='mouse':
        pass
        #dfFullStat['level_value_corr'][i] = dfFullStat['level_value'][i].copy()
    elif  molecule_name in mouse_molecules:
        #print('mouse_molecules')
        dfFullStat['level_value_corr'][i] = dfFullStat['level_value'][i].copy() * dfFullStat['50%'][dfFullStat['idOrg']==molecule_name + '#' + 'mouse'].iloc[0] / dfFullStat['50%'][i]
    elif (molecule_name in rat_molecules) and (dfFullStat['Organism'][i]=='rat'):
        #print(molecule_name) # this case never happens
        pass
        #dfFullStat['level_value_corr'][i] = dfFullStat['level_value'][i].copy()
    elif (molecule_name in rat_molecules) and (dfFullStat['Organism'][i]!='rat'):
        
        #print('rat')
        #print('#')  # this case never happens
        dfFullStat['level_value_corr'][i] = dfFullStat['level_value'][i].copy() * dfFullStat['50%'][dfFullStat['idOrg']==molecule_name + '#' + 'rat'].iloc[0] / dfFullStat['50%'][i]
    else:
        #dfFullStat['level_value_corr'][i] = dfFullStat['level_value'][i].copy() 
        print('not corrected ')  # this case never happens
        pass
    
# for common median
molecular_name_to_mouse_constant_common = np.median(dfFullStat['50%'][dfFullStat['Organism']=='mouse'])
molecular_name_to_rat_constant_common = np.median(dfFullStat['50%'][dfFullStat['Organism']=='rat'])

dfFullStat['level_value_corr_comm'] = dfFullStat['level_value'].copy()  #"NaN"


for i,molecule_name in enumerate(dfFullStat['molecule_name']):
    if dfFullStat['Organism'][i]=='mouse':
        pass
        #dfFullStat['level_value_corr'][i] = dfFullStat['level_value'][i].copy()
    elif  molecule_name in mouse_molecules:
        #print('mouse_molecules')
        dfFullStat['level_value_corr_comm'][i] = dfFullStat['level_value'][i].copy() * molecular_name_to_mouse_constant_common / dfFullStat['50%'][i]
    elif (molecule_name in rat_molecules) and (dfFullStat['Organism'][i]=='rat'):
        #print(molecule_name) # this case never happens
        pass
        #dfFullStat['level_value_corr'][i] = dfFullStat['level_value'][i].copy()
    elif (molecule_name in rat_molecules) and (dfFullStat['Organism'][i]!='rat'):
        
        #print('rat')
        #print('#')  # this case never happens
        dfFullStat['level_value_corr_comm'][i] = dfFullStat['level_value'][i].copy() * molecular_name_to_rat_constant_common / dfFullStat['50%'][i]
    else:
        #dfFullStat['level_value_corr'][i] = dfFullStat['level_value'][i].copy() 
        #print('not corrected ')  # this case never happens
        pass


dfBCmethods = dfBC.groupby(['molecule_name','Organism','Methods'])['level_value'].describe().reset_index()
    
dfBCmethods['id'] = dfBCmethods['molecule_name']+'#'+dfBCmethods['Organism']
dfBCmethodsC1 = pd.merge(dfBCmethods, distrOrganismCorr4[['id', 'level_value','mean_y', 'std','min_y', '25%_y', '50%_y', '75%_y', 'max_y', 'level_value_corr']], how = 'left', on='id')
dfBCmethodsC1['id_OM'] = dfBCmethodsC1['id']+'#'+dfBCmethodsC1['Methods']

ms_molecules = set([dfBCmethodsC1['molecule_name'][i] for i in range(len(dfBCmethodsC1)) if 'mass spectrometry ' in dfBCmethodsC1['Methods'][i]])
review_molecules = set([dfBCmethodsC1['molecule_name'][i] for i in range(len(dfBCmethodsC1)) if 'review' in dfBCmethodsC1['Methods'][i]  ])
db_molecules = set([dfBCmethodsC1['molecule_name'][i] for i in range(len(dfBCmethodsC1)) if 'database' in dfBCmethodsC1['Methods'][i]])
model_molecules = set([dfBCmethodsC1['molecule_name'][i] for i in range(len(dfBCmethodsC1)) if 'model' in dfBCmethodsC1['Methods'][i]])
estimation_molecules = set([dfBCmethodsC1['molecule_name'][i] for i in range(len(dfBCmethodsC1)) if 'estimation' in dfBCmethodsC1['Methods'][i]])

chem_molecules = set([dfBCmethodsC1['molecule_name'][i] for i in range(len(dfBCmethodsC1)) if 'chemistry' in dfBCmethodsC1['Methods'][i]])
mrs_molecules = set([dfBCmethodsC1['molecule_name'][i] for i in range(len(dfBCmethodsC1)) if 'mrs' in dfBCmethodsC1['Methods'][i]])
qa_molecules = set([dfBCmethodsC1['molecule_name'][i] for i in range(len(dfBCmethodsC1)) if 'quantitative autoradiography' in dfBCmethodsC1['Methods'][i]])
    
for i,molecule_name in enumerate(dfBCmethodsC1['id']):
    if dfBCmethodsC1['Methods'][i]=='mass spectrometry ':
        dfBCmethodsC1['level_value_corr_OM'][i] = dfBCmethodsC1['level_value_corr'][i].copy()
    elif  molecule_name in mouse_molecules:
        dfBCmethodsC1['level_value_corr_OM'][i] = dfBCmethodsC1['level_value_corr'][i].copy() * dfBCmethodsC1['50%_y'][dfBCmethodsC1['id']==molecule_name + '#' + 'mouse'].iloc[0] / dfBCmethodsC1['50%_y'][i]
    elif molecule_name in rat_molecules:
        dfBCmethodsC1['level_value_corr_OM'][i] = dfBCmethodsC1['level_value_corr'][i].copy() * dfBCmethodsC1['50%_y'][dfBCmethodsC1['id']==molecule_name + '#' + 'rat'].iloc[0] / dfBCmethodsC1['50%_y'][i]
    else:
        print('not corrected ',i)

    